https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html



Uncommend and run the following pip & python commands when running a new compute for the <b> first</b> time! 

In [122]:
#!python -m spacy download el_core_news_sm
#!pip install pyarrow --upgrade
#!pip install openpyxl
#!pip install xlrd

In [123]:
import spacy
import el_core_news_sm
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [124]:
from spacy.lang.el import GreekLemmatizer

In [125]:
from spacy.lang.el import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

In [126]:
nlp =spacy.load('el_core_news_sm', disable=['tagger', 'parser', 'ner'])


In [127]:
lemmatizer = GreekLemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [128]:
p1 = re.compile('δεν απαντ.{1,3}\s{0,1}',re.IGNORECASE)
p2 = re.compile('\sδα\s',re.IGNORECASE)
p3 = re.compile('δε.{0,1}\s.{0,3}\s{0,1}βρ.{1,2}κ.\s{0,1}',re.IGNORECASE)
p4 = re.compile('[^\d]?\d{10}')
p5 = re.compile('[^\d]?\d{18}|[^\d]\d{20}')
p6 = re.compile('δε[ ν]{0,1} (επιθυμ[α-ω]{2,4}?|ηθελ[α-ω]{1,3}?|θελ[α-ω]{1,4}|.{0,10}ενδιαφερ[α-ω]{2,4})',re.IGNORECASE)
p7 = re.compile('δε[ ν]{0,1} (μπορ[α-ω]{2,5}|.εχει)',re.IGNORECASE)

In [129]:
def loadStopWords():
    sWords = open('stopWords.txt','r',encoding='utf-8')
    sw = set(sWords.read().split('\n'))
    #sw = sw.remove('μη')
    sWords.close()
    return sw

In [130]:
def replaceTerm(text):
    text = p5.sub(' λογαριασμός ',text)
    text = p4.sub(' τηλεφωνο ',text)
    text = p6.sub(' δενθελειδενενδιαφερεται ',text)
    text = p7.sub(' δενεχειδενμπορει ',text)
    text = text.replace('-banking','banking')
    text = text.replace('v banking','vbanking')
    text = text.replace('e banking','ebanking')
    text = text.replace('follow up','followup')
    text = text.replace('safe drive','safedrive')
    text = text.replace('safe pocket','safepocket')
    text = text.replace('sweet home','sweethome')
    text = text.replace('credit card','creditcard')
    text = text.replace('debit card','debitcard')
    text = text.replace('life cycle','lifecycle')
    text = text.replace('eξασφαλιζω','εξασφαλιζω')
    text = text.replace('π/κ','πκ')
    text = text.replace('td','πκ')
    text = text.replace('α/κ','ακ')
    text = text.replace('δ/α','δεναπαντα ')
    #τδ
    text = p1.sub(' δεναπαντα ',text)
    text = p2.sub(' δεναπαντα ',text)
    text = p3.sub(' δεντονβρηκα ',text)
    return text


In [131]:
#sw = nlp.Defaults.stop_words
#sw = sw|{'εχω','απο','ωστε'}
sw = loadStopWords()
def remove_ton(text):
    diction = {'ά':'α','έ':'ε','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   
def clean_text(text):
     #text to string
    text = str(text).lower()
    text = replaceTerm(text)
    
   # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # lower text
    text = [remove_ton(x) for x in text]
    # remove stop words
    text = [x for x in text if x not in sw]
 
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in {'quot','amp'}]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    # remove words with only one letter
    text = " ".join([t for t in text if len(t) > 1])
     # lemmatize text
    text = " ".join([lemmatizer(t.text,t.pos_)[0] for t in nlp(text)])
   
    return(text)

In [132]:
def correct(x,corDict):
    if x in corDict.keys():
        y = corDict[x]
    else:
        y = x
    return y    

In [133]:
def get_ngrams(idf,mindf,minngram,maxngram):
    tfidf = TfidfVectorizer(min_df = mindf,ngram_range = (minngram,maxngram))
    tfidf_result = tfidf.fit_transform(idf['tokenized']).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = [str(x) for x in tfidf_df.columns]
    df_i = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()
    df_i = df_i[df_i['value']>0]
    return df_i

In [134]:
def cleanComments(df):
    df = df[['CON_ROW_ID','CON_COMMENTS']]
    df['tokenized'] = df['CON_COMMENTS'].apply(clean_text)
    df = df.fillna('N/A')
    df['variable'] = df['tokenized'].str.split()
    return df

In [135]:
def getTokens(df):
    df = cleanComments(df)
    df_f = df.explode('variable')[['CON_ROW_ID','variable']]
    return df_f


In [162]:
def getTokencount(df_f,minCount):
    tokenCount = df_f['variable'].value_counts().to_dict()
    df_f['value'] = df_f['variable'].map(tokenCount)
    df_f=df_f[df_f['value']>minCount] 
    return df_f

In [215]:
txt = 'ΟΚ ΕΠΙΚΟΙΝΩΝΙΑ ΣΤΟ 2810 289068...ΔΕΝ ΕΧΕΙ ΝΑ ΠΛΗΡΩΣΕΙ ΤΟ ΚΙΝΗΤΟ ΓΙ ΑΥΤΟ Η ΦΡΑΓΗ...ΑΝΑΜΕΝΕΙ ΑΠΟ ΤΟΝ 08/2019 ΑΠΟΦΑΣΗ ΓΙΑ ΕΠΙΔΟΜΑ ΑΝΑΠΗΡΙΑΣ...ΜΕΧΡΙ 02/2020 ΘΑ ΚΑΤΑΒΛΗΘΟΥΝ ΣΕ ΛΟΓ/ΣΜΟ ΤΗΣ EUROBANK ΤΑ ΑΝΑΔΡΟΜΙΚΑ ΟΠΟΤΕ ΚΑΙ ΘΑ ΤΑΚΤΟΠΟΙΗΘΟΥΝ ΟΛΑ ΤΑ ΛΗΞΗΠΡΟΘΕΣΜΑ...ΔΕΝ ΥΠΑΡΧΕΙ ΔΥΝΑΤΟΤΗΤΑ ΑΛΛΗΣ ΚΤΘΣ ΓΙΑ 12/2019..ΟΚ ΕΝΗΜΕΡΩΣΗ ΚΑΙ ΓΙΑ ΔΙΚΑΙΟΛΟΓΗΤΙΚΑ ΡΥΘΜΙΣΗΣ...ΘΑ ΤΑ ΠΡΟΣΚΟΜΙΣΕΙ ΚΑΠΟΙΟ ΠΑΙΔΙ!!!!/////////////ΟΚ ΚΑΤΑΒΟΛΗ 02/01/2020'

In [216]:
com = {'CON_ROW_ID':[1],'CON_COMMENTS':[txt]}

In [217]:
df = pd.DataFrame(com)

In [218]:
df.head()

,CON_ROW_ID,CON_COMMENTS
0,1,ΟΚ ΕΠΙΚΟΙΝΩΝΙΑ ΣΤΟ 2810 289068...ΔΕΝ ΕΧΕΙ ΝΑ Π...


In [219]:
df = cleanComments(df)

In [220]:
df.head()

,CON_ROW_ID,CON_COMMENTS,tokenized,variable
0,1,ΟΚ ΕΠΙΚΟΙΝΩΝΙΑ ΣΤΟ 2810 289068...ΔΕΝ ΕΧΕΙ ΝΑ Π...,οκ επικοινωνια πληρωσει κινητο φραγη ... αναμε...,"[οκ, επικοινωνια, πληρωσει, κινητο, φραγη, ......"


In [221]:
df_f = getTokens(df)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [222]:
df_f.head()

,CON_ROW_ID,variable
0,1,οκ
0,1,επικοινωνια
0,1,πληρωσει
0,1,κινητο
0,1,φραγη


In [223]:
    tokenCount = df_f['variable'].value_counts().to_dict()
    

In [224]:
df_f.head()

,CON_ROW_ID,variable
0,1,οκ
0,1,επικοινωνια
0,1,πληρωσει
0,1,κινητο
0,1,φραγη


In [225]:
df_f['value'] = df_f['variable'].map(tokenCount)


In [226]:
df_f.head()

,CON_ROW_ID,variable,value
0,1,οκ,1
0,1,επικοινωνια,1
0,1,πληρωσει,1
0,1,κινητο,1
0,1,φραγη,1


In [227]:
df_f=df_f[df_f['value']>0] 

In [228]:
df_f.head()

,CON_ROW_ID,variable,value
0,1,οκ,1
0,1,επικοινωνια,1
0,1,πληρωσει,1
0,1,κινητο,1
0,1,φραγη,1


In [229]:
df_f.head()

,CON_ROW_ID,variable,value
0,1,οκ,1
0,1,επικοινωνια,1
0,1,πληρωσει,1
0,1,κινητο,1
0,1,φραγη,1


In [230]:
corDict = dict(pd.read_excel("corTokens.xls").to_dict("split")['data'])

In [231]:
df_f['token'] = df_f['variable'].apply(lambda x : correct(x,corDict))

In [232]:
df_f.head()

,CON_ROW_ID,variable,value,token
0,1,οκ,1,ok
0,1,επικοινωνια,1,επικοινωνια
0,1,πληρωσει,1,πληρωμη
0,1,κινητο,1,κινητο
0,1,φραγη,1,φραγη


In [233]:
df_f = df_f[df_f['token'] !='rmv']

In [234]:
df_f.head()

,CON_ROW_ID,variable,value,token
0,1,οκ,1,ok
0,1,επικοινωνια,1,επικοινωνια
0,1,πληρωσει,1,πληρωμη
0,1,κινητο,1,κινητο
0,1,φραγη,1,φραγη


In [235]:
df_f = df_f[df_f['token'].str.len() >1]

In [236]:
df_f.head()

,CON_ROW_ID,variable,value,token
0,1,οκ,1,ok
0,1,επικοινωνια,1,επικοινωνια
0,1,πληρωσει,1,πληρωμη
0,1,κινητο,1,κινητο
0,1,φραγη,1,φραγη


In [237]:
df_f = df_f.fillna('N/A')

In [238]:
df_f.head()

,CON_ROW_ID,variable,value,token
0,1,οκ,1,ok
0,1,επικοινωνια,1,επικοινωνια
0,1,πληρωσει,1,πληρωμη
0,1,κινητο,1,κινητο
0,1,φραγη,1,φραγη


In [239]:
df_f = df_f[['CON_ROW_ID','token']].drop_duplicates()

In [240]:
df_f.head()

,CON_ROW_ID,token
0,1,ok
0,1,επικοινωνια
0,1,πληρωμη
0,1,κινητο
0,1,φραγη
